# Data Pipeline

# Goal
Create a data pipeline for the Census and Covid-19

# Methodology
- Clean the data with the foundings in Exploration notebooks
- Concatenate and create the complete dataset for census data. This will be one dataset available on S3.
- This data may be upload to Redshift to do different queries.

As the COVID data is found by city we are going to do multiple aggregations for each city
- Range ages by city
- Gender proportion by city
- People by city
- Adequate access to public services by city
- Internet services by city
- Health quality service by city
- Life expectancy by city

The idea is to combine the COVID data with the census which is how the city sii and get proportion acording to the particularities of each to estimate mortality rates by city, vulnerable cities, "older" cities, connection between health services and covid deaths, internet as a proxy of information about covid and some things like that, changes in life expectancy.

## Sections
1. [**Requirements**](#Requirements)
2. [**Functions**](#Functions)
3. [**Inputs**](#Inputs)
4. [**Pipeline**](#Pipeline)
    - [**Indicators**](#Indicators)
    - [**Intention_features**](#Intention_features)
    - [**TopicEncodings**](#TopicEncodings)
    - [**EngagingFollowsEngaged**](#EngagingFollowsEngaged)
    - [**Hashtags**](#Hashtags)
    - [**Domain**](#Domain)
    - [**Language**](#Language)
    - [**Media**](#Media)
    - [**Links**](#Links)
    - [**Tweet_type**](#Tweet_type)
    - [**Timestamp_features**](#Timestamp_features)
    - [**Followers_and_Followings_features**](#Followers_and_Followings_features)
    - [**Quantile_Discretizer**](#Quantile_Discretizer)
    - [**Intentions_join**](#Intentions_join)
5. [**FeatureSelection**](#FeatureSelection)
6. [**Imputation**](#Imputation)
7. [**Validation**](#Validation)
8. [**Saving_df**](#Saving_df)

# Requirements

In [1]:
#installing packages
sc.install_pypi_package("pandas")
sc.install_pypi_package("boto3")
sc.setCheckpointDir('hdfs:///covid')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1591125667695_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.4-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

  Using cached boto3-1.13.21-py2.py3-none-any.whl (128 kB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached botocore-1.16.21-py2.py3-none-any.whl (6.2 MB)
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
  Using cached urllib3-1.25.9-py2.py3-none-any.whl (126 kB)

In [3]:
import time
import os
import boto3
import gc
import sys
import numpy as np
import pandas as pd
import pickle
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import (FloatType, DateType, StructType, StructField, StringType, LongType, 
    IntegerType, ArrayType, BooleanType, DoubleType)
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StandardScaler, QuantileDiscretizer
gc.enable()

spark = SparkSession.builder.config("spark.sql.shuffle.partitions", 200).appName("covid").getOrCreate()
print(spark.sparkContext.getConf().get('spark.driver.memory'))
print(spark.sparkContext.getConf().get("spark.sql.shuffle.partitions"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2048M
1000

# Functions

## Loading data

In [4]:
def build_schema_census(source="vivienda"):
    """
    Build schema for different sources

    Parameters:
    -----------
    source : str
        Table source may be: "VIV", "PER", "HOG", "FALL", "MGN"

    Return:
    -------
    schema : spark.schema
        Spark schema for loading source table
    """
    if source == "VIV":
        schema = StructType([StructField("TIPO_REG", IntegerType()),
                             StructField("U_DPTO", IntegerType()),
                             StructField("U_MPIO", IntegerType()),
                             StructField("UA_CLASE", IntegerType()),
                             StructField("U_EDIFICA", IntegerType()),
                             StructField("COD_ENCUESTAS", IntegerType()),
                             StructField("U_VIVIENDA", IntegerType()),
                             StructField("UVA_ESTATER", IntegerType()),
                             StructField("UVA1_TIPOTER", LongType()),
                             StructField("UVA2_CODTER", LongType()),
                             StructField("UVA_ESTA_AREAPROT", IntegerType()),
                             StructField("UVA1_COD_AREAPROT", LongType()),
                             StructField("UVA_USO_UNIDAD", IntegerType()),
                             StructField("V_TIPO_VIV", LongType()),
                             StructField("V_CON_OCUP", LongType()),
                             StructField("V_TOT_HOG", LongType()),
                             StructField("V_MAT_PARED", LongType()),
                             StructField("V_MAT_PISO", LongType()),
                             StructField("VA_EE", LongType()),
                             StructField("VA1_ESTRATO", LongType()),
                             StructField("VB_ACU", LongType()),
                             StructField("VC_ALC", LongType()),
                             StructField("VD_GAS", LongType()),
                             StructField("VE_RECBAS", LongType()),
                             StructField("VE1_QSEM", LongType()),
                             StructField("VF_INTERNET", LongType()),
                             StructField("V_TIPO_SERSA", LongType()),
                             StructField("L_TIPO_INST", LongType()),
                             StructField("L_EXISTEHOG", LongType()),
                             StructField("L_TOT_PERL", LongType())
                             ])
    elif source == "HOG":
        schema = StructType([StructField("TIPO_REG", IntegerType()),
                             StructField("U_DPTO", IntegerType()),
                             StructField("U_MPIO", IntegerType()),
                             StructField("UA_CLASE", IntegerType()),
                             StructField("COD_ENCUESTAS", IntegerType()),
                             StructField("U_VIVIENDA", IntegerType()),
                             StructField("H_NROHOG", LongType()),
                             StructField("H_NRO_CUARTOS", LongType()),
                             StructField("H_NRO_DORMIT", LongType()),
                             StructField("H_DONDE_PREPALIM", LongType()),
                             StructField("H_AGUA_COCIN", LongType()),
                             StructField("HA_NRO_FALL", LongType()),
                             StructField("HA_TOT_PER", LongType())
                             ])
    elif source == "PER":
        schema = StructType([StructField("TIPO_REG", IntegerType()),
                             StructField("U_DPTO", IntegerType()),
                             StructField("U_MPIO", IntegerType()),
                             StructField("UA_CLASE", IntegerType()),
                             StructField("U_EDIFICA", IntegerType()),
                             StructField("COD_ENCUESTAS", IntegerType()),
                             StructField("U_VIVIENDA", IntegerType()),
                             StructField("P_NROHOG", LongType()),
                             StructField("P_NRO_PER", IntegerType()),
                             StructField("P_SEXO", IntegerType()),
                             StructField("P_EDADR", IntegerType()),
                             StructField("P_PARENTESCOR", LongType()),
                             StructField("PA1_GRP_ETNIC", IntegerType()),
                             StructField("PA11_COD_ETNIA", LongType()),
                             StructField("PA12_CLAN", LongType()),
                             StructField("PA21_COD_VITSA", LongType()),
                             StructField("PA22_COD_KUMPA", LongType()),
                             StructField("PA_HABLA_LENG", LongType()),
                             StructField("PA1_ENTIENDE", LongType()),
                             StructField("PB_OTRAS_LENG", LongType()),
                             StructField("PB1_QOTRAS_LENG", LongType()),
                             StructField("PA_LUG_NAC", IntegerType()),
                             StructField("PA_VIVIA_5ANOS", LongType()),
                             StructField("PA_VIVIA_1ANO", LongType()),
                             StructField("P_ENFERMO", LongType()),
                             StructField("P_QUEHIZO_PPAL", LongType()),
                             StructField("PA_LO_ATENDIERON", LongType()),
                             StructField("PA1_CALIDAD_SERV", LongType()),
                             StructField("CONDICION_FISICA", LongType()),
                             StructField("P_ALFABETA", LongType()),
                             StructField("PA_ASISTENCIA", LongType()),
                             StructField("P_NIVEL_ANOSR", LongType()),
                             StructField("P_TRABAJO", LongType()),
                             StructField("P_EST_CIVIL", LongType()),
                             StructField("PA_HNV", LongType()),
                             StructField("PA1_THNV", LongType()),
                             StructField("PA2_HNVH", LongType()),
                             StructField("PA3_HNVM", LongType()),
                             StructField("PA_HNVS", LongType()),
                             StructField("PA1_THSV", LongType()),
                             StructField("PA2_HSVH", LongType()),
                             StructField("PA3_HSVM", LongType()),
                             StructField("PA_HFC", LongType()),
                             StructField("PA1_THFC", LongType()),
                             StructField("PA2_HFCH", LongType()),
                             StructField("PA3_HFCM", LongType()),
                             StructField("PA_UHNV", LongType()),
                             StructField("PA1_MES_UHNV", LongType()),
                             StructField("PA2_ANO_UHNV", LongType())
                             ])
    elif source == "FALL":
        schema = StructType([StructField("TIPO_REG", IntegerType()),
                             StructField("U_DPTO", IntegerType()),
                             StructField("U_MPIO", IntegerType()),
                             StructField("UA_CLASE", IntegerType()),
                             StructField("COD_ENCUESTAS", IntegerType()),
                             StructField("U_VIVIENDA", IntegerType()),
                             StructField("F_NROHOG", IntegerType()),
                             StructField("FA1_NRO_FALL", IntegerType()),
                             StructField("FA2_SEXO_FALL", IntegerType()),
                             StructField("FA3_EDAD_FALL", IntegerType()),
                             StructField("FA4_CERT_DEFUN", IntegerType())
                             ])
    elif source == "MGN":
        schema = StructType([StructField("U_DPTO", IntegerType()),
                             StructField("U_MPIO", IntegerType()),
                             StructField("UA_CLASE", IntegerType()),
                             StructField("UA1_LOCALIDAD", IntegerType()),
                             StructField("U_SECT_RUR", IntegerType()),
                             StructField("U_SECC_RUR", IntegerType()),
                             StructField("UA2_CPOB", IntegerType()),
                             StructField("U_SECT_URB", IntegerType()),
                             StructField("U_SECC_URB", IntegerType()),
                             StructField("U_MZA", IntegerType()),
                             StructField("U_EDIFICA", IntegerType()),
                             StructField("COD_ENCUESTAS", IntegerType()),
                             StructField("U_VIVIENDA", IntegerType())
                             ])
    else:
        print("Source not valid. Enter one of the following sources: VIV, PER, HOG, FALL, MGN")
    return schema


def build_schema_covid(source="covid"):
    """
    Build schema for different covid sources

    Parameters:
    -----------
    source : str
        Table source may be: "covid", "tests"

    Return:
    -------
    schema : spark.schema
        Spark schema for loading source table
    """
    if source == "covid":
        schema = StructType([StructField("fecha_de_notificaci_n", DateType()),
                             StructField("c_digo_divipola", StringType()),
                             StructField("ciudad_de_ubicaci_n", StringType()),
                             StructField("departamento", StringType()),
                             StructField("atenci_n", StringType()),
                             StructField("edad", IntegerType()),
                             StructField("sexo", StringType()),
                             StructField("tipo", StringType()),
                             StructField("estado", StringType()),
                             StructField("pa_s_de_procedencia", StringType()),
                             StructField("fis", DateType()),
                             StructField("fecha_diagnostico", DateType()),
                             StructField("fecha_recuperado", DateType()),
                             StructField("fecha_reporte_web", DateType()),
                             StructField("tipo_recuperaci_n", StringType()),
                             StructField("codigo_departamento", StringType()),
                             StructField("codigo_pais", StringType()),
                             StructField("pertenencia_etnica", StringType()),
                             StructField("nombre_grupo_etnico", StringType()),
                             StructField("fecha_de_muerte", DateType()),
                             StructField("Asintomatico", IntegerType()),
                             StructField("divipola_dpto", IntegerType()),
                             StructField("divipola_mpio", IntegerType())
                             ])
    elif source == "tests":
        schema = StructType([StructField("fecha", DateType()),
                             StructField("acumuladas", LongType()),
                             StructField("amazonas", LongType()),
                             StructField("antioquia", LongType()),
                             StructField("arauca", LongType()),
                             StructField("atlantico", LongType()),
                             StructField("bogota", LongType()),
                             StructField("bolivar", LongType()),
                             StructField("boyaca", LongType()),
                             StructField("caldas", LongType()),
                             StructField("caqueta", LongType()),
                             StructField("casanare", LongType()),
                             StructField("cauca", LongType()),
                             StructField("cesar", LongType()),
                             StructField("choco", LongType()),
                             StructField("cordoba", LongType()),
                             StructField("cundinamarca", LongType()),
                             StructField("guainia", LongType()),
                             StructField("guajira", LongType()),
                             StructField("guaviare", LongType()),
                             StructField("huila", LongType()),
                             StructField("magdalena", LongType()),
                             StructField("meta", LongType()),
                             StructField("narino", LongType()),
                             StructField("norte_de_santander", LongType()),
                             StructField("putumayo", LongType()),
                             StructField("quindio", LongType()),
                             StructField("risaralda", LongType()),
                             StructField("san_andres", LongType()),
                             StructField("santander", LongType()),
                             StructField("sucre", LongType()),
                             StructField("tolima", LongType()),
                             StructField("valle_del_cauca", LongType()),
                             StructField("vaupes", LongType()),
                             StructField("vichada", LongType()),
                             StructField("procedencia_desconocida", LongType()),
                             StructField("positivas_acumuladas", LongType()),
                             StructField("negativas_acumuladas", LongType()),
                             StructField("positividad_acumulada", LongType()),
                             StructField("indeterminadas", LongType()),
                             StructField("barranquilla", LongType()),
                             StructField("cartagena", LongType()),
                             StructField("santa_marta", LongType())
                             ])
    else:
        print("Source not valid. Enter one of the following sources: 'covid', 'tests'")
    return schema
              
def build_schema_divipola(source="divipola"):
    """
    Build schema for different covid sources

    Parameters:
    -----------
    source : str
        Table source may be: "covid", "tests"

    Return:
    -------
    schema : spark.schema
        Spark schema for loading source table
    """
    if source == "divipola":
        schema = StructType([StructField("cod_depto", IntegerType()),
                             StructField("cod_mpio", IntegerType()),
                             StructField("dpto", StringType()),
                             StructField("nom_mpio", StringType()),
                             StructField("tipo_municipio", StringType())
                             ])
    else:
        print("Source not valid. Enter one of the following sources: 'covid', 'tests'")
    return schema
              
def get_censo_paths(bucket_s3, directory_key):
    """
    Get dictionary of census data for each department
    
    Parameters:
    -----------
    bucket_s3 : s3.Bucket
        Boto3 Bucket object
    directory_key : path
        Directory key in S3
    
    Return:
    -------
    dict_paths_departments : dict
        Dictionary with the data path for each departtment
    """
    dict_paths_departments = {}
    for object_summary in bucket_s3.objects.filter(Prefix=directory_key):
        name = object_summary.key
        if name.endswith(".CSV"):
            list_paths = name.split("/")
            department = list_paths[2].split("_")[1]
            if "MGN" in list_paths[-1]:
                if not(department in dict_paths_departments):
                    dict_paths_departments[department] = {}
                dict_paths_departments[department].update({"MGN": os.path.join(f"s3a://{bucket_s3.name}", name)})                
            elif "FALL" in list_paths[-1]:
                if not(department in dict_paths_departments):
                    dict_paths_departments[department] = {}
                dict_paths_departments[department].update({"FALL": os.path.join(f"s3a://{bucket_s3.name}", name)})
            elif "HOG" in list_paths[-1]:
                if not(department in dict_paths_departments):
                    dict_paths_departments[department] = {}
                dict_paths_departments[department].update({"HOG": os.path.join(f"s3a://{bucket_s3.name}", name)})
            elif "VIV" in list_paths[-1]:
                if not(department in dict_paths_departments):
                    dict_paths_departments[department] = {}
                dict_paths_departments[department].update({"VIV": os.path.join(f"s3a://{bucket_s3.name}", name)})
            elif "PER" in list_paths[-1]:
                if not(department in dict_paths_departments):
                    dict_paths_departments[department] = {}
                dict_paths_departments[department].update({"PER": os.path.join(f"s3a://{bucket_s3.name}", name)})
    return dict_paths_departments

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Preprocessing

In [5]:



def mediaCounter(row, media='Photo'):
    counter=0
    if type(row)==list:
        for elem in row:
            if elem==media:
                counter+=1
    else:
        pass
    return counter

def listCounter(row):
    counter=0
    if type(row)==list:
        counter+=len(row)
    else:
        pass
    return counter

def labelEncoder(row, mapping_encode):
    """
    Label Encoding or Array<String> types
    
    
    Parameters:
    -----------
    row : list(string)
        List of string or labels
    mapping_encode : dict(label, integer)
        Encoding of some top K labels
    Return:
    -------
    out : list(integers)
        List of Label Encoders.
        if not in mapping Encoded to len(map)
        if not a list Encoded to len(map)+1
    """
    out=[]
    if type(row)==list:
        for elem in row:
            if elem in mapping_encode:
                out.append(mapping_encode.get(elem))
            else:
                out.append(len(mapping_encode))
    else:
        out.append(len(mapping_encode)+1)
    return out

def labelEncoderSingle(row, mapping_encode):
    out=[]
    if row:
        if row in mapping_encode:
            out.append(mapping_encode.get(row))
        else:
            out.append(len(mapping_encode))
    else:
        out.append(len(mapping_encode)+1)
    return out

def hashtagSumCounter(row, mapping_hashtag_count):
    counter=0
    if type(row)==list:
        for elem in row:
            if elem in mapping_hashtag_count:
                counter+=mapping_hashtag_count.get(elem, 0)
    else:
        pass
    return counter

def get_distribution_array_col(df, col):
    distribution_df = df.select(col).filter(F.col(col).isNotNull())\
                              .withColumn(col, 
                                          F.explode(F.col(col)))\
                              .groupBy(col).count()\
                              .orderBy(F.col("count").desc())
    return distribution_df

def save_pkl_to_s3(obj, key_filename, bucket_name):
    serialized_obj = pickle.dumps(obj)
    s3 = boto3.client('s3')
    s3.put_object(Bucket=bucket_name, Key=key_filename, 
                  Body=serialized_obj)
    
def columns2cast(df):
    columns = []
    for col in df.schema:
        if col.dataType.typeName()=="array":
            columns.append(col)
    return columns
    
def cast_array2string(df, columns):
    for col in columns:
        df = df.withColumn(col.name, F.col(col.name).cast(StringType()))
    return df

def cast_string2array(df, columns):
    for col in columns:
        df= df.withColumn(col, 
                          F.split(F.regexp_replace(F.col(col), r"(^\[)|(\]$)|(')", ""),
                                  ", "))
    return df
    
def mappings(df, col, top_k):
    col_dist = get_distribution_array_col(df, col)
    df_col_dist = col_dist.limit(top_k)
    df_col = df_col_dist.toPandas().rename(columns={'_1': col, 
                                                    '_2': 'count'})\
                                    .reset_index().set_index(col)
    mapping_encode = df_col['index'].to_dict()
    mapping_count = df_col['count'].to_dict()
    return mapping_encode, mapping_count

def mapping_label_encoder(df, col, top_k):
    col_dist = df.select(col).filter(F.col(col).isNotNull())\
                      .groupBy(col).count()\
                      .orderBy(F.col("count").desc())
    df_col_dist = col_dist.limit(top_k)
    df_col = df_col_dist.toPandas().rename(columns={'_1': col, 
                                                    '_2': 'count'})\
                                    .reset_index().set_index(col)
    mapping_encoder = df_col['index'].to_dict()
    return mapping_encoder

def validator(df):
    columns_w_nan = {}
    for col in df.schema:
        null_count = df.filter(F.col(col.name).isNull()).count()
        if null_count>0:
            columns_w_nan[col.name]=null_count
    return columns_w_nan

# Mappings
tweet_type_mapping = {'TopLevel':0, 'Quote':1, 'Retweet':2, 'Reply':3}

# UDF SQL
PhotoCounter_udf = F.udf(lambda row: mediaCounter(row, 'Photo'), 
                         IntegerType())
VideoCounter_udf = F.udf(lambda row: mediaCounter(row, 'Video'), 
                         IntegerType())
GifCounter_udf = F.udf(lambda row: mediaCounter(row, 'GIF'), 
                         IntegerType())
listCounter_udf = F.udf(listCounter, 
                         IntegerType())
tweet_encoded_udf = F.udf(lambda x: tweet_type_mapping[x], 
                             IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Inputs

In [223]:
metadata = {"CENSO": {"VIVIENDA": {"useful_columns": ['U_DPTO', 'U_MPIO', 'UA_CLASE', 'U_EDIFICA',
                                                      'COD_ENCUESTAS', 'U_VIVIENDA', 'UVA_USO_UNIDAD',
                                                      'V_TIPO_VIV', 'V_CON_OCUP', 'V_TOT_HOG',
                                                      'V_MAT_PARED', 'V_MAT_PISO', 'VA_EE', 'VA1_ESTRATO', 'VB_ACU', 'VC_ALC',
                                                      'VD_GAS', 'VE_RECBAS', 'VE1_QSEM', 'VF_INTERNET', 'V_TIPO_SERSA',
                                                      'L_TIPO_INST', 'L_EXISTEHOG', 'L_TOT_PERL']
                                   },
                      "HOGAR": {"useful_columns": ['U_DPTO', 'U_MPIO', 'UA_CLASE', 'COD_ENCUESTAS',
                                                   'U_VIVIENDA', 'H_NROHOG', 'H_NRO_CUARTOS', 'H_NRO_DORMIT',
                                                   'H_DONDE_PREPALIM', 'H_AGUA_COCIN', 'HA_NRO_FALL', 'HA_TOT_PER']},
                      "PERSONAS": {"useful_columns": ['U_DPTO', 'U_MPIO', 'UA_CLASE', 'U_EDIFICA',
                                                      'COD_ENCUESTAS', 'U_VIVIENDA', 'P_NROHOG', 'P_NRO_PER', 'P_SEXO',
                                                      'P_EDADR', 'P_PARENTESCOR', 'PA_LUG_NAC',
                                                      'PA_VIVIA_5ANOS', 'PA_VIVIA_1ANO', 'P_ENFERMO', 'P_QUEHIZO_PPAL',
                                                      'PA_LO_ATENDIERON', 'PA1_CALIDAD_SERV', 'CONDICION_FISICA',
                                                      'P_ALFABETA', 'PA_ASISTENCIA', 'P_NIVEL_ANOSR', 'P_TRABAJO',
                                                      'P_EST_CIVIL', 'PA_HNV', 'PA1_THNV', 'PA2_HNVH', 'PA3_HNVM', 'PA_HNVS',
                                                      'PA1_THSV', 'PA2_HSVH', 'PA3_HSVM', 'PA_HFC', 'PA1_THFC', 'PA2_HFCH',
                                                      'PA3_HFCM']},
                      "FALLECIDOS": {"useful_columns": ['U_DPTO', 'U_MPIO', 'UA_CLASE', 'COD_ENCUESTAS',
                                                        'U_VIVIENDA', 'F_NROHOG', 'FA1_NRO_FALL', 'FA2_SEXO_FALL',
                                                        'FA3_EDAD_FALL', 'FA4_CERT_DEFUN']},
                      "GEOREFERENCIACION": {"useful_columns": ['U_DPTO', 'U_MPIO', 'UA_CLASE', 'UA1_LOCALIDAD', 'U_SECT_RUR',
                                                               'U_SECC_RUR', 'UA2_CPOB', 'U_SECT_URB', 'U_SECC_URB', 'U_MZA',
                                                               'U_EDIFICA', 'COD_ENCUESTAS', 'U_VIVIENDA']},
                      "DIVIPOLA": {"useful_columns": ['cod_depto', 'cod_mpio', 'dpto', 'nom_mpio', 'tipo_municipio']}
                      },
            }

bucket='censo-covid'
s3_resource = boto3.resource('s3')
bucket_s3 = s3_resource.Bucket(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Paths**

In [ ]:
censo_covid_bucket_s3 = f"s3a://{bucket}"

raw_data_path = os.path.join(censo_covid_bucket_s3, "raw-data")
censo_data_path = os.path.join(raw_data_path, "censo")
covid_tests_path = os.path.join(raw_data_path, "covid-tests.csv")
covid_path = os.path.join(raw_data_path, "covid.csv")
divipola_path = os.path.join(raw_data_path, "divipola.csv")

dict_paths_departments = get_censo_paths(bucket_s3, directory_key=os.path.join("raw-data", "censo"))

In [227]:
# Test
if len(list(bucket_s3.objects.filter(Prefix=f"{data_path}/engaging-users-test", Delimiter='./')))==0:
    df = parse_data(test_path, has_labels=False).repartition(200)
    engaging_users_test = df.select("engaging_user_id").distinct()
    engaging_users_test.write.csv(engaging_users_test_path)

engaging_users_test = spark.read.csv(engaging_users_test_path, 
                                    schema=StructType([StructField('engaging_user_id', StringType())]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Pipeline

### Load_data

In [ ]:
covid_data = spark.read.csv(covid_path, 
                            schema=build_schema_covid(source="covid"))
covid_tests_data = spark.read.csv(covid_tests_path, 
                                  schema=build_schema_covid(source="tests"))
divipola_data = spark.read.csv(divipola_path, 
                                  schema=build_schema_divipola(source="divipola"))

In [ ]:
census_data_dict={}
for department, val in dict_paths_departments.items():
    for table, table_path in val.items():
        if not(table in census_data_dict):
            census_data_dict[table] = spark.read.csv(table_path, schema=build_schema_census(source=table))
        else:
            aux = spark.read.csv(table_path, schema=build_schema_census(source=table))
            census_data_dict[table] = census_data_dict[table].union(aux)

### Clean data

In [ ]:
drop_viv_cols = [col for col in census_data_dict["VIV"].columns\
                 if col not in metadata["CENSO"]["VIVIENDA"]["useful_columns"]]
census_data_dict["VIV"] = census_data_dict["VIV"].drop(drop_viv_cols)

In [ ]:
drop_hog_cols = [col for col in census_data_dict["HOG"].columns\
                 if col not in metadata["CENSO"]["HOGAR"]["useful_columns"]]
census_data_dict["HOG"] = census_data_dict["HOG"].drop(drop_hog_cols)
census_data_dict["HOG"] = census_data_dict["HOG"].fillna(99, subset=['H_NROHOG'])
census_data_dict["HOG"] = census_data_dict["HOG"].withColumn("H_NROHOG", F.col("H_NROHOG").cast(IntegerType()))

In [ ]:
drop_per_cols = [col for col in census_data_dict["PER"].columns\
                 if col not in metadata["CENSO"]["PERSONAS"]["useful_columns"]]
census_data_dict["PER"] = census_data_dict["PER"].drop(drop_per_cols)
census_data_dict["PER"] = census_data_dict["PER"].fillna(99, subset=['P_NROHOG'])
census_data_dict["PER"] = census_data_dict["PER"].withColumn("P_NROHOG", F.col("P_NROHOG").cast(IntegerType()))

In [ ]:
drop_fall_cols = [col for col in census_data_dict["FALL"].columns\
                 if col not in metadata["CENSO"]["FALLECIDOS"]["useful_columns"]]
census_data_dict["FALL"] = census_data_dict["FALL"].drop(drop_fall_cols)
census_data_dict["FALL"] = census_data_dict["FALL"].fillna(99, subset=['F_NROHOG'])
census_data_dict["FALL"] = census_data_dict["FALL"].withColumn("F_NROHOG", F.col("F_NROHOG").cast(IntegerType()))

In [ ]:
drop_mgn_cols = [col for col in census_data_dict["MGN"].columns\
                 if col not in metadata["CENSO"]["GEOREFERENCIACION"]["useful_columns"]]
census_data_dict["MGN"] = census_data_dict["MGN"].drop(drop_mgn_cols)

### JoinTables

In [ ]:
df_per_complete = census_data_dict["PER"].join(census_data_dict["VIV"], 
                                               on=['U_DPTO', 'U_MPIO', 'UA_CLASE', 
                                                   'U_EDIFICA', 'COD_ENCUESTAS',
                                                   'U_VIVIENDA'],
                                               how="left")

In [ ]:
assert df_per_complete.filter(F.col("UVA_USO_UNIDAD").isNull()).count() == 0

In [ ]:
df_per_complete = df_per_complete.merge(census_data_dict["HOG"], 
                                        how="left", 
                                        left_on=['U_DPTO', 'U_MPIO', 'UA_CLASE', 'COD_ENCUESTAS', 'U_VIVIENDA', 'P_NROHOG'], 
          right_on=['U_DPTO', 'U_MPIO', 'UA_CLASE', 'COD_ENCUESTAS', 'U_VIVIENDA', 'H_NROHOG'])

In [231]:
if submission:
    print("Submission")
elif test:
    print("Test")
else:
    if training:
        print("Train")
        df = train
    else:
        print("Validation")
        df = val    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test

In [232]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12434838

In [233]:
columns_w_nan = validator(df)
print(columns_w_nan)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'hashtags': 9802258, 'present_media': 7855555, 'present_links': 10560235, 'present_domains': 10560235}

### Indicators

In [234]:
if not(submission) and not(test):
    df = df.withColumn('indicator_reply',F.when(F.col('reply_timestamp').isNotNull(), 1).otherwise(0))
    df = df.withColumn('indicator_retweet',F.when(F.col('retweet_timestamp').isNotNull(), 1).otherwise(0))
    df = df.withColumn('indicator_retweet_with_comment',
                       F.when(F.col('retweet_with_comment_timestamp').isNotNull(),1).otherwise(0))
    df = df.withColumn('indicator_like', F.when(F.col('like_timestamp').isNotNull(),1).otherwise(0))
    df = df.withColumn('indicator_interaction', 
                       F.when(F.col('indicator_reply')+\
                              F.col('indicator_retweet')+\
                              F.col('indicator_retweet_with_comment')+\
                              F.col('indicator_like')>0, 1)\
                       .otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Intention_features

In [235]:
if training:
    if len(list(bucket_s3.objects.filter(Prefix=f"{data_path}/intentions-"+suffix_sample, Delimiter='./')))==0:
        print("Creating Intention")
        intention_df = df.select("engaging_user_id", "indicator_reply", "indicator_retweet", 
                                    "indicator_retweet_with_comment", "indicator_like", "indicator_interaction")\
                        .groupBy("engaging_user_id").agg(F.sum(F.col("indicator_interaction")).alias("n_interactions"), 
                                                         F.sum(F.col("indicator_retweet_with_comment"))\
                                                         .alias("n_commented"),
                                                         F.sum(F.col("indicator_like")).alias("n_liked"),
                                                         F.sum(F.col("indicator_reply")).alias("n_replied"),
                                                         F.sum(F.col("indicator_retweet")).alias("n_retweeted"),
                                                         F.count(F.col("indicator_interaction"))\
                                                         .alias("total_appearance"))
        columns = ['n_interactions', 'n_commented', 'n_liked', 'n_replied', 'n_retweeted']
        for col_i in columns:
            intention_df = intention_df.withColumn("perc_" + col_i, F.col(col_i)/(F.col("total_appearance")))
        intention_df = intention_df.drop(*columns)
        join_users_not_test = join_users_not_test.select(F.col("engaging_user_id").alias("drop_users"))
        join_users_not_test = join_users_not_test.sample(withReplacement=False,
                                                         fraction=0.15,
                                                         seed=42)
        intention_df = intention_df.join(join_users_not_test, 
                                         intention_df.engaging_user_id==join_users_not_test.drop_users, 
                                         how="left_anti").drop("drop_users")
        intention_df.repartition(1000).write.csv(intentions_path)
    else:
        print("Intention already created")
schema = StructType([StructField('engaging_user_id', StringType()),
             StructField('total_appearance', LongType()),
             StructField('perc_n_interactions', DoubleType()),
             StructField('perc_n_commented', DoubleType()),
             StructField('perc_n_liked', DoubleType()),
             StructField('perc_n_replied', DoubleType()),
             StructField('perc_n_retweeted', DoubleType())])
intention_df = spark.read.csv(intentions_path, schema=schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Hashtags

In [236]:
if training:
    print("Creating hashtags features")
    mapping_hashtag_encode, mapping_hashtag_count = mappings(df, "hashtags", top_k_hashtags)
    # Saving pkl
    save_pkl_to_s3(mapping_hashtag_encode, key_hashtag_mapping, bucket)
    save_pkl_to_s3(mapping_hashtag_count, key_hashtag_count, bucket)

# Load dict mapping language
mapping_hashtag_encode = pickle.loads(s3_resource.Bucket(bucket).Object(key_hashtag_mapping).get()['Body'].read())
mapping_hashtag_count = pickle.loads(s3_resource.Bucket(bucket).Object(key_hashtag_count).get()['Body'].read())
        
hashtagsEncoder_udf = F.udf(lambda x: labelEncoder(x, mapping_hashtag_encode), 
                         StringType())
df = df.withColumn('hashtagEncoded', hashtagsEncoder_udf(df.hashtags))
hashtagSumCounter_udf = F.udf(lambda x: hashtagSumCounter(x, mapping_hashtag_count), 
                             IntegerType())
df = df.withColumn('hashtagSumCount', hashtagSumCounter_udf(df['hashtags']))
df = df.withColumn('hashtagCount', listCounter_udf(df.hashtags))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Domain

In [237]:
if training:
    print("Creating domains features")
    mapping_domain_encode, mapping_domain_count = mappings(df, "present_domains", top_k_domains)
    # Saving pkl
    save_pkl_to_s3(mapping_domain_encode, key_domain_mapping, bucket)
    save_pkl_to_s3(mapping_domain_count, key_domain_count, bucket)

# Load dict mapping language
mapping_domain_encode = pickle.loads(s3_resource.Bucket(bucket).Object(key_domain_mapping).get()['Body'].read())

domainEncoder_udf = F.udf(lambda x: labelEncoder(x, mapping_domain_encode), 
                         StringType())
df = df.withColumn('domainEncoded', domainEncoder_udf(df.present_domains))
df = df.withColumn('domainCount', listCounter_udf(df.present_domains))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Language

In [238]:
if training:
    print("Creating language features")
    mapping_encoder = mapping_label_encoder(df, "language", top_k_languages)
    save_pkl_to_s3(mapping_encoder, key_language_mapping, bucket)

# Load dict mapping language
mapping_language_encode = pickle.loads(s3_resource.Bucket(bucket).Object(key_language_mapping).get()['Body'].read())
        
languageEncoder_udf = F.udf(lambda x: labelEncoderSingle(x, mapping_language_encode)[0], 
                         StringType())
df = df.withColumn('languageEncoded', languageEncoder_udf(df.language))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Media

In [239]:
dict_media_counter={'PhotoCount': PhotoCounter_udf,
                    'VideoCount': VideoCounter_udf, 
                    'GIFCount': GifCounter_udf}
for media, media_fun in dict_media_counter.items():
    df = df.withColumn(media, media_fun(df.present_media))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Links

In [240]:
df = df.withColumn('linkCount', listCounter_udf(df.present_links))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Tweet_type

In [241]:
df = df.withColumn('tweetEncoded', tweet_encoded_udf(df.tweet_type))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Timestamp_features

#### Date features

In [242]:
timestamp_feats = [i for i in df.columns if (('timestamp' in i) or ('creation' in i))]

# Timestamp to dates
for col_ts in timestamp_feats:
    # Taking only the preffix of each column
    preffix = col_ts.split("_timestamp")[0]
    df = df.withColumn(preffix + "_date", F.from_unixtime(col_ts, 'yyyy-MM-dd HH:mm:ss'))

# From tweet_date extracting day of week, week of month and hour of tweet
df = df.withColumn( 'tweet_timestamp_day_of_week', F.date_format(F.col('tweet_date'), 'u') )
df = df.withColumn( 'tweet_timestamp_week_of_month',  F.date_format(F.col('tweet_date'), "W" ) )
df = df.withColumn( 'tweet_timestamp_hour',  F.date_format(F.col('tweet_date'), "H" ) )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Scaling_features
As standard scaling will be done manually, we have to create some dictionaries to save the information of the scaling.

In [243]:
if training: 
    scaling_dict = dict()
else:
    scaling_dict = pickle.loads(s3_resource.Bucket(bucket).Object(key_scaling_features).get()['Body'].read())
    assert type(scaling_dict) == dict

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [244]:
#Seconds from engagee account creation to tweet posting
# Differences of timestamps return a result inv Seconds
df = df.withColumn( 'tweet_timestamp_to_engagee_account_creation',  
                   F.col('tweet_timestamp') - F.col('engaged_with_user_account_creation'))
if training: 
    mean_col, sttdev_col = df.select(F.mean('tweet_timestamp_to_engagee_account_creation'),
                                     F.stddev('tweet_timestamp_to_engagee_account_creation')).first()
    # Saving scaling features
    scaling_dict['tweet_timestamp_to_engagee_account_creation'] = { 'mean': mean_col, 'std': sttdev_col}    
mean_col = scaling_dict['tweet_timestamp_to_engagee_account_creation']['mean']    
sttdev_col = scaling_dict['tweet_timestamp_to_engagee_account_creation']['std']
    
df = df.withColumn('tweet_timestamp_to_engagee_account_creation'+'_ss', 
                   (F.col('tweet_timestamp_to_engagee_account_creation') - mean_col) / (2*sttdev_col))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [245]:
#Seconds from engaging account creation to tweet posting
# Differences of timestamps return a result inv Seconds
df = df.withColumn( 'tweet_timestamp_to_engaging_account_creation',  
                   F.col('tweet_timestamp') - F.col('engaging_user_account_creation'))
#Scaling
if training: 
    mean_col, sttdev_col = df.select(F.mean('tweet_timestamp_to_engaging_account_creation'), 
                                     F.stddev('tweet_timestamp_to_engaging_account_creation')).first()
    # Saving scaling features
    scaling_dict['tweet_timestamp_to_engaging_account_creation'] = { 'mean': mean_col, 'std': sttdev_col}    
mean_col = scaling_dict['tweet_timestamp_to_engaging_account_creation']['mean']    
sttdev_col = scaling_dict['tweet_timestamp_to_engaging_account_creation']['std']

df = df.withColumn('tweet_timestamp_to_engaging_account_creation'+'_ss', 
                   (F.col('tweet_timestamp_to_engaging_account_creation') - mean_col) / (2*sttdev_col))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Followers_and_Followings_features

In [246]:
df = df.withColumn('engaged_with_vs_engaging_follower_diff',
                   F.col('engaged_with_user_follower_count') - F.col('engaging_user_follower_count'))
df = df.withColumn('engaged_with_vs_engaging_following_diff', 
                   F.col('engaged_with_user_following_count') - F.col('engaging_user_following_count'))
df = df.withColumn('engaged_follow_diff',
                   F.col('engaged_with_user_follower_count') - F.col('engaged_with_user_following_count'))
df = df.withColumn('engaging_follow_diff', 
                   F.col('engaging_user_follower_count') - F.col('engaging_user_following_count'))
df = df.withColumn('engaged_follower_diff_engaging_following', 
                   F.col('engaged_with_user_follower_count') - F.col('engaging_user_following_count'))
df = df.withColumn('engaged_following_diff_engaging_follower', 
                   F.col('engaged_with_user_following_count') - F.col('engaging_user_follower_count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [247]:
columns = ["engaged_with_vs_engaging_follower_diff", "engaged_with_vs_engaging_following_diff", 
           "engaged_follow_diff", "engaging_follow_diff", "engaged_follower_diff_engaging_following", 
           "engaged_following_diff_engaging_follower"]
if training:
    diff_min_dict = {}
    for col in columns:
        diff_min_dict[col] = df.select(F.min(col)).first()[0]
    save_pkl_to_s3(diff_min_dict, key_diff_min, bucket)
        
diff_min_dict = pickle.loads(s3_resource.Bucket(bucket).Object(key_diff_min).get()['Body'].read())
for col in columns:
    df = df.withColumn(col, F.col(col)-diff_min_dict[col])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Log and Standard Scaling of followers and difference features

In [248]:
# Followers/following counts and differences will transformed to logarithm. We are choosing the column names.
columns_to_log = ['engaged_with_vs_engaging_follower_diff', 'engaged_with_vs_engaging_following_diff', 
                    'engaged_follow_diff', 'engaging_follow_diff', 
                    'engaged_follower_diff_engaging_following', 'engaged_following_diff_engaging_follower', 
                    'engaged_with_user_follower_count', 'engaging_user_follower_count', 
                    'engaged_with_user_following_count', 'engaging_user_following_count']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [249]:
for column in columns_to_log:
    df = df.withColumn(column + '_log', F.log(F.col(column)+1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [250]:
cols_to_scale = [column for column in df.columns if ('_log' in column)]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [251]:
if training:
    for col in cols_to_scale:
        mean_col, sttdev_col = df.select(F.mean(col), F.stddev(col)).first()
        scaling_dict[col] = { 'mean': mean_col, 'std': sttdev_col}

for col in cols_to_scale:
    mean_col = scaling_dict[col]['mean']
    sttdev_col = scaling_dict[col]['std']
    df = df.withColumn(col+'_ss', (F.col(col) - mean_col) / (2*sttdev_col))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Quantile_Discretizer

In [252]:
if training:
    qds_obj = {}
    qds_paths
    for col in qds_paths.keys():
        qds = QuantileDiscretizer(numBuckets=50, inputCol=col, outputCol=col+"_q")
        qds = qds.fit(df)
        qds.save(qds_paths[col])
        
for col in qds_paths.keys():
    qds = QuantileDiscretizer.load(qds_paths[col])
    df = qds.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Intentions_join

In [253]:
df = df.join(intention_df, 
             on="engaging_user_id", 
             how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [254]:
columns = ["perc_n_interactions", 'perc_n_commented', 'perc_n_liked', 'perc_n_replied', 'perc_n_retweeted']
if training:
    dict_mean_perc = {}
    for col in columns:
        dict_mean_perc[col] = intention_df.select(F.mean(col)).first()[0]
    save_pkl_to_s3(dict_mean_perc, key_impute_perc, bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### NumScale

In [255]:
num_cols_to_scale = ["hashtagSumCount", "hashtagCount",
                     "domainCount", "PhotoCount",
                     "VideoCount", "GIFCount",
                     "linkCount", "total_appearance",
                     "perc_n_interactions",
                     "perc_n_commented",
                     "perc_n_liked",
                     "perc_n_replied",
                     "perc_n_retweeted"]
#Scaling
if training:
    for col in num_cols_to_scale:
        mean_col, sttdev_col = df.select(F.mean(col), F.stddev(col)).first()
        scaling_dict[col] = {'mean': mean_col, 'std': sttdev_col}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Imputation

#### DiffsImputer

In [256]:
columns = ['engaged_with_vs_engaging_follower_diff_log_ss', 
           'engaged_with_vs_engaging_following_diff_log_ss', 
           'engaged_follow_diff_log_ss', 
           'engaging_follow_diff_log_ss', 
           'engaged_follower_diff_engaging_following_log_ss', 
           'engaged_following_diff_engaging_follower_log_ss']
for col in columns:
    df = df.withColumn(col, F.when(F.col(col).isNotNull(), F.col(col)).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### IntentionsImputer

In [257]:
df = df.withColumn('total_appearance', F.when(F.col("total_appearance").isNotNull(), 
                                                   F.col("total_appearance"))\
                          .otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [258]:
columns = ["perc_n_interactions", 'perc_n_commented', 'perc_n_liked', 'perc_n_replied', 'perc_n_retweeted']
        
dict_mean_perc = pickle.loads(s3_resource.Bucket(bucket).Object(key_impute_perc).get()['Body'].read())
for col in columns:
    df = df.withColumn(col, F.when(F.col(col).isNotNull(), F.col(col))\
                       .otherwise(dict_mean_perc[col]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Scaling after imputed

In [259]:
for col in num_cols_to_scale:
    mean_col = scaling_dict[col]['mean']
    sttdev_col = scaling_dict[col]['std']
    df = df.withColumn(col+'_ss', (F.col(col) - mean_col) / (2*sttdev_col))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Clipping

In [260]:
deviations_to_clip = 5.0
for col in num_cols_to_scale:
    df = df.withColumn(col+'_ss', F.when(F.abs(F.col(col+'_ss'))<5, 
                                         F.col(col+'_ss'))\
                       .otherwise(deviations_to_clip))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Saving scaling_dict

In [261]:
if training: 
    # Saving scaling dictionary to pickle
    save_pkl_to_s3(scaling_dict, key_scaling_features, bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# FeatureSelection

In [262]:
original = ['text_tokens', 'tweet_id', 'engaged_with_user_id', 'engaged_with_user_is_verified',
            'engaging_user_id', 'engaging_user_is_verified', 'engagee_follows_engager']

processed = ['hashtagEncoded', 'hashtagSumCount_ss', 'hashtagCount_ss', #hashtags
             'domainEncoded', 'domainCount_ss', #domains
             'tweetEncoded', # tweet type encoded
             'languageEncoded',# language encoded
             'tweet_timestamp_day_of_week', 'tweet_timestamp_week_of_month', 'tweet_timestamp_hour', #tweet timestamp
             'tweet_timestamp_to_engagee_account_creation_ss', #engagee time in twitter proxy
             'tweet_timestamp_to_engaging_account_creation_ss', #engaging time in twitter proxy
             'engaged_with_vs_engaging_follower_diff_log_ss', #followers engaged
             'engaged_with_vs_engaging_following_diff_log_ss', #followings engaged
             'engaged_follow_diff_log_ss', #diff follows engaged
             'engaging_follow_diff_log_ss', #diff follows engaging
             'engaged_follower_diff_engaging_following_log_ss',  #followings engaging
             'engaged_following_diff_engaging_follower_log_ss', #followers engaging
             'engaged_with_user_follower_count_log_ss', #engaged follower count
             'engaging_user_follower_count_log_ss', #engaging follower count
             'engaged_with_user_following_count_log_ss',  #engaged following count
             'engaging_user_following_count_log_ss', #engaging following count
             'PhotoCount_ss', 'VideoCount_ss', 'GIFCount_ss', 'linkCount_ss', #media
             'engaged_with_user_follower_count_q', 'engaged_with_user_following_count_q', #Quantile
             'engaged_with_user_account_creation_q', 'engaging_user_follower_count_q', #Quantile
             'engaging_user_following_count_q', 'engaging_user_account_creation_q', #Quantile
             'total_appearance_ss', 'perc_n_interactions_ss', 'perc_n_commented_ss', 
             'perc_n_liked_ss', 'perc_n_replied_ss', 'perc_n_retweeted_ss' #intentions
             ]

labels = ['indicator_reply', 'indicator_retweet', 'indicator_retweet_with_comment',
          'indicator_like', 'indicator_interaction']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [263]:
if submission:
    cols_to_select = original + processed
elif test:
    cols_to_select = original + processed
else:
    cols_to_select = original + processed + labels

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [264]:
new_df = df.select(cols_to_select)
columns = columns2cast(new_df)
new_df = cast_array2string(new_df, columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [265]:
dict_types = {'bool': ['engaged_with_user_is_verified', 'engaging_user_is_verified', 'engagee_follows_engager'],
              'id': ['tweet_id', 'engaged_with_user_id', 'engaging_user_id'], 
              'num': ['hashtagSumCount_ss', 'hashtagCount_ss', 
                      'domainCount_ss', 
                      'tweet_timestamp_to_engagee_account_creation_ss', #engagee time in twitter proxy
                      'tweet_timestamp_to_engaging_account_creation_ss', #engaging time in twitter proxy
                      'engaged_with_vs_engaging_follower_diff_log_ss', #followers engaged
                      'engaged_with_vs_engaging_following_diff_log_ss', #followings engaged
                      'engaged_follow_diff_log_ss', #diff follows engaged
                      'engaging_follow_diff_log_ss', #diff follows engaging
                      'engaged_follower_diff_engaging_following_log_ss',  #followings engaging
                      'engaged_following_diff_engaging_follower_log_ss', #followers engaging
                      'engaged_with_user_follower_count_log_ss', #engaged follower count
                      'engaging_user_follower_count_log_ss', #engaging follower count
                      'engaged_with_user_following_count_log_ss',  #engaged following count
                      'engaging_user_following_count_log_ss', #engaging following count
                      'PhotoCount_ss', 'VideoCount_ss', 'GIFCount_ss', 'linkCount_ss', #media
                      'total_appearance_ss', 'perc_n_interactions_ss', 'perc_n_commented_ss', 
                      'perc_n_liked_ss', 'perc_n_replied_ss', 'perc_n_retweeted_ss' #intentions
                     ], 
              'cat': ['tweetEncoded', 'languageEncoded',
                      'tweet_timestamp_day_of_week', 'tweet_timestamp_week_of_month', 'tweet_timestamp_hour',
                      'engaged_with_user_follower_count_q', 'engaged_with_user_following_count_q', #Quantile
                      'engaged_with_user_account_creation_q', 'engaging_user_follower_count_q', #Quantile
                      'engaging_user_following_count_q', 'engaging_user_account_creation_q', #Quantile
                     ], 
              'ors': ['text_tokens'], 
              'unors':['hashtagEncoded', 'domainEncoded']}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [266]:
for key, val in dict_types.items():
    for column in val:
        new_df = new_df.withColumnRenamed(column, column + '_' + key)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [267]:
new_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12434838

In [129]:
#stats = ["mean", "50%", "min", "max"]
#new_df.select("hashtagSumCount_ss_num", 
#              "perc_n_retweeted_ss_num", 
#              "total_appearance_ss_num").summary(*stats).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------+-----------------------+-----------------------+
|summary|hashtagSumCount_ss_num|perc_n_retweeted_ss_num|total_appearance_ss_num|
+-------+----------------------+-----------------------+-----------------------+
|   mean|  -0.03561698913424515|   -0.02871487470125...|   -0.13855573076697658|
|    50%|  -0.08201654219284023|   -0.07379754563317474|    -0.2589475275080297|
|    min|  -0.08201654219284023|   -0.31412261503645766|    -0.4152207742939386|
|    max|                   5.0|     1.6672083509634679|                    5.0|
+-------+----------------------+-----------------------+-----------------------+

# MergeUserBucket

In [268]:
map_user_bucket = spark.read.csv(map_user_bucket_path, schema= StructType([StructField('user_id', StringType()),
                                                                           StructField('final_bucket', IntegerType())]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [269]:
new_df = new_df.join(map_user_bucket, new_df.engaged_with_user_id_id==map_user_bucket.user_id, how="left")
new_df = new_df.drop("user_id")
new_df = new_df.withColumnRenamed("final_bucket", "engaged_with_user_id_bucket")

new_df = new_df.join(map_user_bucket, new_df.engaging_user_id_id==map_user_bucket.user_id, how="left")
new_df = new_df.drop("user_id")
new_df = new_df.withColumnRenamed("final_bucket", "engaging_user_id_bucket")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [270]:
new_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[text_tokens_ors: string, tweet_id_id: string, engaged_with_user_id_id: string, engaged_with_user_is_verified_bool: boolean, engaging_user_id_id: string, engaging_user_is_verified_bool: boolean, engagee_follows_engager_bool: boolean, hashtagEncoded_unors: string, hashtagSumCount_ss_num: double, hashtagCount_ss_num: double, domainEncoded_unors: string, domainCount_ss_num: double, tweetEncoded_cat: int, languageEncoded_cat: string, tweet_timestamp_day_of_week_cat: string, tweet_timestamp_week_of_month_cat: string, tweet_timestamp_hour_cat: string, tweet_timestamp_to_engagee_account_creation_ss_num: double, tweet_timestamp_to_engaging_account_creation_ss_num: double, engaged_with_vs_engaging_follower_diff_log_ss_num: double, engaged_with_vs_engaging_following_diff_log_ss_num: double, engaged_follow_diff_log_ss_num: double, engaging_follow_diff_log_ss_num: double, engaged_follower_diff_engaging_following_log_ss_num: double, engaged_following_diff_engaging_follower_log_ss_num: doub

# Validation

In [271]:
columns_w_nan = validator(new_df)
print(columns_w_nan)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{}

# Saving_df

In [272]:
print(processed_submission_path)
print(processed_test_path)
print(processed_train_path)
print(processed_val_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://twitter-challenge-factored/final-data/processed/submission-final-complete
s3a://twitter-challenge-factored/final-data/processed/test-final-complete
s3a://twitter-challenge-factored/final-data/processed/train-final-complete
s3a://twitter-challenge-factored/final-data/processed/val-final-complete

In [273]:
processed_submission_path = f"hdfs:///submission-{suffix_sample}"
processed_test_path = f"hdfs:///test-{suffix_sample}"
processed_train_path = f"hdfs:///train-{suffix_sample}"
processed_val_path = f"hdfs:///val-{suffix_sample}"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [274]:
if submission:
    new_df.write.option("header","true").csv(processed_submission_path)
    print("Submission saved")
elif test:
    new_df.write.option("header","true").csv(processed_test_path)
    print("Test saved")
elif training:
    new_df.write.option("header","true").csv(processed_train_path)
    print("Train saved")
else:
    new_df.write.option("header","true").csv(processed_val_path)
    print("Valid saved")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test saved

# REPLACE FOR S3-DIST-CP
s3-dist-cp --src hdfs:///test-final-complete --dest s3://twitter-challenge-factored/final-data/processed/test-final-complete
s3-dist-cp --src hdfs:///test-final-complete --dest s3://twitter-challenge-factored/final-data/processed/test-final-complete
s3-dist-cp --src hdfs:///test-final-complete --dest s3://twitter-challenge-factored/final-data/processed/test-final-complete
s3-dist-cp --src hdfs:///test-final-complete --dest s3://twitter-challenge-factored/final-data/processed/test-final-complete

In [168]:
new_df.schema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType(List(StructField(text_tokens_ors,StringType,true),StructField(tweet_id_id,StringType,true),StructField(engaged_with_user_id_id,StringType,true),StructField(engaged_with_user_is_verified_bool,BooleanType,true),StructField(engaging_user_id_id,StringType,true),StructField(engaging_user_is_verified_bool,BooleanType,true),StructField(engagee_follows_engager_bool,BooleanType,true),StructField(hashtagEncoded_unors,StringType,true),StructField(hashtagSumCount_ss_num,DoubleType,true),StructField(hashtagCount_ss_num,DoubleType,true),StructField(domainEncoded_unors,StringType,true),StructField(domainCount_ss_num,DoubleType,true),StructField(tweetEncoded_cat,IntegerType,true),StructField(languageEncoded_cat,StringType,true),StructField(tweet_timestamp_day_of_week_cat,StringType,true),StructField(tweet_timestamp_week_of_month_cat,StringType,true),StructField(tweet_timestamp_hour_cat,StringType,true),StructField(tweet_timestamp_to_engagee_account_creation_ss_num,DoubleType,true),StructFie

In [ ]:
StructType([StructField("text_tokens_ors",StringType),
            StructField("tweet_id_id",StringType),
            StructField("engaged_with_user_id_id",StringType),
            StructField("engaged_with_user_is_verified_bool",BooleanType),
            StructField("engaging_user_id_id",StringType),
            StructField("engaging_user_is_verified_bool",BooleanType),
            StructField("engagee_follows_engager_bool",BooleanType),
            StructField("hashtagEncoded_unors",StringType),
            StructField("hashtagSumCount_num",IntegerType),
            StructField("hashtagCount_num",IntegerType),
            StructField("domainEncoded_unors",StringType),
            StructField("domainCount_num",IntegerType),
            StructField("tweetEncoded_cat",IntegerType),
            StructField("languageEncoded_cat",StringType),
            StructField("tweet_timestamp_day_of_week_cat",StringType),
            StructField("tweet_timestamp_week_of_month_cat",StringType),
            StructField("tweet_timestamp_hour_cat",StringType),
            StructField("tweet_timestamp_to_engagee_account_creation_ss_num",DoubleType),
            StructField("tweet_timestamp_to_engaging_account_creation_ss_num",DoubleType),
            StructField("engaged_with_vs_engaging_follower_diff_log_ss_num",DoubleType),
            StructField("engaged_with_vs_engaging_following_diff_log_ss_num",DoubleType),
            StructField("engaged_follow_diff_log_ss_num",DoubleType),
            StructField("engaging_follow_diff_log_ss_num",DoubleType),
            StructField("engaged_follower_diff_engaging_following_log_ss_num",DoubleType),
            StructField("engaged_following_diff_engaging_follower_log_ss_num",DoubleType),
            StructField("engaged_with_user_follower_count_log_ss_num",DoubleType),
            StructField("engaging_user_follower_count_log_ss_num",DoubleType),
            StructField("engaged_with_user_following_count_log_ss_num",DoubleType),
            StructField("engaging_user_following_count_log_ss_num",DoubleType),
            StructField("PhotoCount_num",IntegerType),
            StructField("VideoCount_num",IntegerType),
            StructField("GIFCount_num",IntegerType),
            StructField("linkCount_num",IntegerType),
            StructField("engaged_with_user_follower_count_q_cat",DoubleType),
            StructField("engaged_with_user_following_count_q_cat",DoubleType),
            StructField("engaged_with_user_account_creation_q_cat",DoubleType),
            StructField("engaging_user_follower_count_q_cat",DoubleType),
            StructField("engaging_user_following_count_q_cat",DoubleType),
            StructField("engaging_user_account_creation_q_cat",DoubleType),
            StructField("total_appearance_num",LongType),
            StructField("perc_n_interactions_num",DoubleType),
            StructField("perc_n_commented_num",DoubleType),
            StructField("perc_n_liked_num",DoubleType),
            StructField("perc_n_replied_num",DoubleType),
            StructField("perc_n_retweeted_num",DoubleType),
            StructField("indicator_reply",IntegerType),
            StructField("indicator_retweet",IntegerType),
            StructField("indicator_retweet_with_comment",IntegerType),
            StructField("indicator_like",IntegerType),
            StructField("indicator_interaction",IntegerType)])